### 5.3 ファイル読み込み (具体例2: SAM形式) 
SAMファイルの読み込みを行う。__SAMファイル__ は、@で始まるヘッダー行と各リードのアライメント情報を格納している (表5.7) アライメント情報の各列は、タブ (\t) で区切られている。  

表5.7 SAMファイルのアライメント情報  

| 列数 | Field | 情報 | 例 (20行目)  |
|--------------|-----------|------|--------|
| 1 | QNAME | リード名 | SRR453566.24 |
| 2 | FLAG | アライメント情報 | 83 |
| 3 | RNAME | リファレンス名 | NC_001139.9 |
| 4 | POS | マッピング位置 | 727620 |
| 5 | MAPQ | マッピングスコア | 60 |
| 6 | CIGAR | マッピングの状況 | 101M |
| 7 | RNEXT | ペアエンドリードのマッピングされたリファレンス名<br>=の場合は、同じリファレンス名 | = |
| 8 | PNEXT | ペアエンドリードのマッピング位置 | 727518 |
| 9 | TLEN  | リード間の距離 | -203 |
| 10 | SEQ | リードの塩基配列 | AAGGGTAA.... |
| 11 | QUAL | 塩基配列のクォリティデータ | ?DCCDDDD ... |

In [8]:
#　ファイルの中身を25行目まで確認する
!head -n 25 ./data/SRR453566.sam  

@HD	VN:1.0	SO:unsorted
@SQ	SN:NC_001133.9	LN:230218
@SQ	SN:NC_001134.8	LN:813184
@SQ	SN:NC_001135.5	LN:316620
@SQ	SN:NC_001136.10	LN:1531933
@SQ	SN:NC_001137.3	LN:576874
@SQ	SN:NC_001138.5	LN:270161
@SQ	SN:NC_001139.9	LN:1090940
@SQ	SN:NC_001140.6	LN:562643
@SQ	SN:NC_001141.2	LN:439888
@SQ	SN:NC_001142.9	LN:745751
@SQ	SN:NC_001143.9	LN:666816
@SQ	SN:NC_001144.5	LN:1078177
@SQ	SN:NC_001145.3	LN:924431
@SQ	SN:NC_001146.8	LN:784333
@SQ	SN:NC_001147.6	LN:1091291
@SQ	SN:NC_001148.4	LN:948066
@SQ	SN:NC_001224.1	LN:85779
@PG	ID:hisat2	PN:hisat2	VN:2.1.0	CL:"/usr/local/pkg/hisat2/2.1.0/hisat2-align-s --wrapper basic-0 -p 4 -x ../reference/hisat/s288c.fna -S SRR453566.sam -1 /tmp/52279.inpipe1 -2 /tmp/52279.inpipe2"
SRR453566.24	83	NC_001139.9	727620	60	101M	=	727518	-203	AAGGGTAAAGCTAAGGGTGATATTCCAGGTGTTAGATTCAAGGTCGTTAAGGTCTCTGGTGTCTCCTTGTTGGCTTTGTGGAAAGAAAAGAAGGAAAAGCC	?DCCDDDDDDDDDDDDDD@CDCCBEEEDFFFFFHHHHFHJJJJIJJJJJJJJIJGJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJHHHHHFFFFFCCC	AS:i:0	XN:i:0	XM:i:0	X

### 5.3.1 ビット演算子  
2列目のFLAG情報は、リードのマッピング状況を知ることができる。各情報は下記のように定義されている (表5.8)
  
Bit とは、、、  
Binary Digit（バイナリ ディジット） の略で、   
0か1だけを表す情報の最小単位です。2進数 (接頭辞 0b で 0b0001などと表現される)  
16進数は 2進数の4桁で1文字を表すのでよく一緒に使われる。(接頭辞 0x で 0x1 などと表現される)

表5.8 SAMファイルのFLAG情報  
| 十進数 |  Bit | 説明 (SAM定義) | Decoding SAM flags |
|--------------|-----------|------|-------|
| 1 | 0x1 | template having multiple seqments in sequencing | read paired |
| 2 | 0x2 | each segment properly aliged according to the aligner | read mapped in proper pair |
| 4 | 0x4 | segment unmapped | read unmapped |
| 8 | 0x8 | next segment in the template unmapped | mate unmapped |
| 16 | 0x10 | SEQ being reverse complemented | read reverse strand |
| 32 | 0x20 | SEQ of the next segment in the template being reverse complemented | mate reverse strand |
| 64 | 0x40 | the first segment in the template | first in pair |
| 128 | 0x80 | the last segment in the template | second in pair |
| 256 | 0x100 | secondary alignment | not primary alignment |
| 512 | 0x200 | not passing filters, such as platform/vendor quality controls | read failes platform/vendor quality checks |
| 1024 | 0x400 | PCR or optical duplicate | read is PCR or optical duplicate |
| 2048 | 0x800 | supplementary alignment | supplementary alignment |


FLAG情報に関しては、Decoding SAM flagsサイト (https://broadinstitute.github.io/picard/explain-flags.html) がわかりやすい。例えば。FLAG83を持つリードは、 read paired (0x1)、read mapped in proper pair (0x2)、read reverse strand (0x10)、first in pair (0x40)でリードがマッピングされていることを示している。いくつかのFLAGについて、bit情報をまとめた。(表5.9)

表5.9 FLAGとbit情報の対応の例  
| 十進数 | Bit | FLAG=83 | FLAG=163 | FLAG=77 | FLAG=137 |
|--------------|-----------|------|------|------|------|
| 1 | 0x1 | ◯ | ◯ | ◯ | ◯ |
| 2 | 0x2 | ◯ | ◯ |   |   |
| 4 | 0x4 |   |   | ◯ |   |
| 8 | 0x8 |   |   | ◯ | ◯ |
| 16 | 0x10 | ◯ |   |   |   |
| 32 | 0x20 |   | ◯ |   |   |
| 64 | 0x40 | ◯ |   | ◯ |   |
| 128 | 0x80 |   | ◯ |   | ◯ |
| 256 | 0x100 |   |   |   |   |
| 512 | 0x200 |   |   |   |   |
| 1024 | 0x400 |   |   |   |   |
| 2048 | 0x800 |   |   |   |   |


ビット演算子 (表5.10) を用いて、FLAG情報を扱うことができる。例えば、if FLAG & 0x4: とすることで、unmap かどうかを判定できる。


表5.10 ビット演算子の例  
| ビット演算子 | 説明 |　例 |
|--------------|-----------|-----|
| x\|y | xとyの論理和 (OR) をとる | x = x \| 0x4  # xの3ビット目をONにする |
| x&y | xとyの論理積(AND)をとる |　if x & 0x4: と記載するとxの3ビット目がON（1）かどうか調べたいときにif文で使います。<br>x = x & ~0x4  # xの3ビット目をOFFにする  |
| x^y | xとyの排他的論理和(XOR)をとる|　x = x ^ 0x4  # xの3ビット目をトグル（1→0, 0→1）|


__FLGの状況の読み込み__  
FLGの状況を確認する。

In [18]:
path = './data/SRR453566.sam'
dic = {}
with open(path) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
        s = line.split('\t') #タブで区切る
        FLG = s[1] #2列目を格納
        if FLG in dic: #すでに辞書にFLGのkeyが格納されているか？
            dic[FLG] += 1
        else:
            dic[FLG] = 1

#出力FLAG, 総数
for k, v in dic.items():
    print(k + '\t' + str(v))

83	197
163	197
99	239
147	239
355	15
403	15
81	3
161	3
65	2
129	2
77	14
141	14
113	1
177	1
89	5
133	7
339	17
419	17
97	2
145	2
153	2
69	3
73	2
137	1


FLAG情報の集計結果が得られた。例えば、FLAG83のリードが197本あったことがわかる。多い順で並べ替えて、出力する  

In [19]:
sorted(dic.items(), key = lambda x:x[1], reverse = True)

[('99', 239),
 ('147', 239),
 ('83', 197),
 ('163', 197),
 ('339', 17),
 ('419', 17),
 ('355', 15),
 ('403', 15),
 ('77', 14),
 ('141', 14),
 ('133', 7),
 ('89', 5),
 ('81', 3),
 ('161', 3),
 ('69', 3),
 ('65', 2),
 ('129', 2),
 ('97', 2),
 ('145', 2),
 ('153', 2),
 ('73', 2),
 ('113', 1),
 ('177', 1),
 ('137', 1)]

__コードの解説__ 
  
| 部分 | 意味 |
|-----|------|
| dic.items() | キーと値のペアのリストを返す |
| sorted() | 並び替えを行う |
| key=lambda x: x[1] | 並び替えの基準を、各ペアの2番目（値）に指定する |
| reverse=True | 大きい順（降順）に並び替える |
  

__特定のFLAGを有するリード情報の読み込み__  
特定のFLAGを有するリード情報を抽出する。マッピングされなかったリードを抽出する。read unmapped は、0x4, 4で表現される。

In [21]:
readList = []
with open(path) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
        s = line.split('\t') #タブで区切る
        
        FLG = int(s[1]) #ファイルから読み込むと str型 なので、これを int に変換する。
        if FLG & 0x4: #FLGの値を2進数にして、16進数の0x4も2進数にすると0b0100、つまり3ビット目ONであれば TRUE
            readList.append(s[0])

#出力FLAG, 総数
for item in readList:
    print(item)

SRR453566.116
SRR453566.116
SRR453566.126
SRR453566.153
SRR453566.153
SRR453566.156
SRR453566.156
SRR453566.178
SRR453566.178
SRR453566.183
SRR453566.242
SRR453566.242
SRR453566.243
SRR453566.243
SRR453566.255
SRR453566.255
SRR453566.263
SRR453566.263
SRR453566.302
SRR453566.310
SRR453566.322
SRR453566.342
SRR453566.342
SRR453566.354
SRR453566.354
SRR453566.350
SRR453566.381
SRR453566.394
SRR453566.394
SRR453566.396
SRR453566.396
SRR453566.409
SRR453566.433
SRR453566.433
SRR453566.472
SRR453566.499
SRR453566.499
SRR453566.503


ビット演算子を用いて、read unmapped (0x4) ビットを持つFLAG69, 77, 133, 141 のリードが抽出された。samtools を使うことによって、同様の処理が可能となる。

__マッピングされなかったリードの読み込み__  
特定のFLAGを有するリードを抽出する。マッピングされなかったリードを抽出する。ペアエンドのリード(Read1, Read2) を別々に抽出する。配列も合わせて抽出する。

In [31]:
readList = []
with open(path) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
            
        s = line.split('\t') #タブで区切る
        FLG = int(s[1]) #ファイルから読み込むと str型 なので、これを int に変換する。
        
        if (FLG & 0x4) and (FLG & 0x40): ## read unmapped (0x4)  first in pair (0x40)
            readList.append([s[0]+'-R1', s[9]]) 

        if(FLG & 0x4) and (FLG & 0x80) : ## read unmapped (0x4)  second in pair (0x40)
            readList.append([s[0]+'-R2', s[9]])

with open ('./output/unmapped.fasta', 'w') as f:
    for item, seq in readList:
        f.write('>'+item+'\n'+seq+'\n') ##Fasta形式

マッピングされなかったリードを抽出し、FASTA形式の "unmapped.fasta"へのファイル出力をした。web blastなどを使うことで、マッピングされなかったリードの由来を調べることができる。

In [32]:
%more ./output/unmapped.fasta

>SRR453566.116-R1
GCCATCTGTGGCACGCTGTAATAGCCAATCTTTTACTTCCCAGATCTTGTCATACCACGGCGCTTCACGAGGTAGTAATGACACTGGTGCTGTTTCTGCAG
>SRR453566.116-R2
ACGATGTGGCCGGGCCTGCAGAAACAGCACCAGTGTCATTACTACCTC
>SRR453566.126-R2
GTGAGTTGGAGCAGCAGAGGTGGGGTTCTTTGGAGCTTCAGGAGAGGGAACTGG
>SRR453566.153-R1
CGGAAGTGTATCGTACAGTAGACGGAGTATACTAGTATAGTCTATAGTCCGTGGAATTCTAAGTGCCAGCTTTATAATGTCATTCTCCTTACTACAGACCC
>SRR453566.153-R2
GAAGGTTTTCGCGGTTTTTATAAACAAAACTTTCGTTACGAAATCGAGCAATCACCCCAGCTGCGTATTTGGAAATTC
>SRR453566.156-R1
CGTCATGTTTATCCTATGTGCGAGCATGGTATCAAGGCCTCATACTGTATGGCCCTTAATGATGCCATGGTGTCGGCTAATGGTAACCTGTATGGACTAGC
>SRR453566.156-R2
TTCGTCTCCCATTGTCCCTCATCCTCACTAAACAGCTTTTCTGCTAGTCCATACAGGTTACCATTAGCCGACACCATGGCATCATTA
>SRR453566.178-R1
CTAAAACTTGCCTAGAGGGGAATAGTATATGTTGAGCAAGTGTGTTGATGTAGTATAGTAAGTCAAATCTAAATTTATATCCAACATATGAAGCTAGACCA
>SRR453566.178-R2
TAGAAATAGATGAAGATGAATACAAAGAGAAGGTACATGAAATGCAAAAGTTGATTGGTCTAGCTTCATATGTTGGATATAAATTTAGATTTGACTTACTA
>SRR453566.183-R2
GGCAAGTCTGAGACCATTCTTCTCTTACTCTCAGAATTG

## 5.4 正規表現  
__正規表現__ (regular expression) とは、「文字列の集合を一つの文字列で表現する方法の一つである」。通常の文字列とメタ文字と呼ばれる特殊な文字を組み合わせてパターンを作り、パターンに指定された法則でならむ文字列の検索を実現できる。正規表現は、ファイルや他のデータの中から、複雑な文字列のパターンを検索するのに役立つ。  
制御配列の一つの __TATAボックス__ を例に、正規表現を試してみる。  
  
TATAボックスは、RNAポリメラーゼⅡによる転写開始位置の上流25塩基対の位置、あるいはさらに上流に存在する共通した塩基配列で、5'=TATA[A/T]AA[G/A]-3'と定義されている。転写因子（例：TFIIDのTBPサブユニット）がTATAボックスに結合することで、RNAポリメラーゼIIを呼び込み、転写開始複合体を形成する。これが転写開始（RNA合成の始まり）の目印になる。
  
TATAボックスを有するリードをSAMファイルから検索する。文字列一致で探索する場合は、4通りの文字列を探索する必要がある。総補佐配列も含めると計8通りの文字列の探索が必要となる。  

  * 5'-TATA[A]AA[G]-3'
  * 5'-TATA[A]AA[A]-3'
  * 5'-TATA[T]AA[A]-3'
  * 5'-TATA[A]AA[G]-3'

正規表現は、標準ライブラリの re モジュールを使う (表5.11)  
  
        表5.11 reモジュールでよく使うメソッド
| メソッド | 説明 |
|---------|-------|
| compile() | 正規表現patternのオブジェクトを生成 |
| matcth() | 文字列の先頭で正規表現patternの有無を探索 |
| search() | 文字列ないで正規表現patternの有無を探索 |
| findall() | 文字列ないですべての正規表現patternを探索 |
| finditer() | 文字列ないで正規表現patternを探索し、取得するイテレータを返す。for文を添えて使用 |
  
正規表現で使用するメタ文字を一部紹介する (表5.12)  
  
      表5.12 正規表現で使用するメタ文字の一部  
| パターン | 説明 |
|---------|-------|
| \d      | 1個の数字 |
| \D      | 1個の数字以外の文字 |
| \w      | 1個の英字 |
| \W      | 1個の英字以外の文字 |
| \s      | 1個の空白文字 |
| \S      | 1個の空白文字以外の文字 |
| .      | \n以外の任意の文字 |
| ^      | 文字列の先頭 |
| $      | 文字列の末尾 |
| *      | 直前の正規表現を0回以上、できるだけ多く繰り返したもの <br> ab*はa, ab, またはaに任意個数のbを続けたもの |
| +      | 直前の正規表現を1回以上、できるだけ多く繰り返したもの <br> ab+はab, またはaに任意個数のbを続けたもの |
| ?      | 直前の正規表現を0回もしくは1回<br>ab?はaあるいはab |
| []      | 文字の集合<br>[abc]はaまたはbまたはc |

__正規表現を用いない探索__  
TATAボックスを有する配列をreモジュールを使わずに探索する。4種類の配列それぞれの有無について調べる。

In [41]:
path = './data/SRR453566.sam'
TATA1 = 'TATAAAAG'
TATA2 = 'TATAAAAA'
TATA3 = 'TATATAAG'
TATA4 = 'TATATAAA'

with open(path) as f:
    for line in f:
        line = line.rstrip()  #改行を削除
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
        s= line.split('\t') #タブで区切る
        sequence = s[9] #10列目を格納

        if TATA1 in sequence:
            print('TATA1:' + sequence)

        if TATA2 in sequence:
            print('TATA2:' + sequence)

        if TATA3 in sequence:
            print('TATA3:' + sequence)

        if TATA4 in sequence:
            print('TATA4:' + sequence)

        

TATA3:AAATCTAAACACACCATCAGGATCAATTTACTGCGCAGTATGTACTCGTACTTGTATATAAGATTCAAAGGATACCAAGAAAATGCTATTACG
TATA4:CGTGCTGATCAGGATATTTCTCTTCTTCATAGCATAGAAACCAAGTTGTTCCCATATATAAACTTCGCAGCCCTAAATAGTGAACAATCTCATAATTTTTG
TATA2:AAATGGCAAAAAAAAAAGTAAAAAATGGCCCAACTGTATGAGACGTATAAAAAACGTGAAGGGTGAAGAAAAGAATGCCACTGCCCAATTTTATGCTTAAT
TATA4:AAGCCGCCAAAATTCAACAGGGTACCGACTTGGCCGAAGTAGCCCCAATATTATGTGCTGGTGTTACTGTATATAAAGCACTAAAAGAGGCAGACTTGAA


4通りのTATAボックスを用意して、それぞれについて探索を行った。そう補佐についても探索が必要な場合は、4通りの配列に加えてそれぞれに対して同様の処理を行えばよい。

__正規表現を用いる探索__  
TATAボックスを有する配列をreモジュールによる正規表現パターンを用いて探索する。  

In [42]:
import re

TATA = re.compile('TATA[AT]AA[GA]') #パターンをコンパイル AorT  GorA

with open(path) as f:
    for line in f:
        line = line.rstrip()  #改行を削除
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
        s= line.split('\t') #タブで区切る
        sequence = s[9] #10列目を格納

        m = TATA.search(sequence) #pattern を検索

        if m:
            print(sequence)
        

AAATCTAAACACACCATCAGGATCAATTTACTGCGCAGTATGTACTCGTACTTGTATATAAGATTCAAAGGATACCAAGAAAATGCTATTACG
CGTGCTGATCAGGATATTTCTCTTCTTCATAGCATAGAAACCAAGTTGTTCCCATATATAAACTTCGCAGCCCTAAATAGTGAACAATCTCATAATTTTTG
AAATGGCAAAAAAAAAAGTAAAAAATGGCCCAACTGTATGAGACGTATAAAAAACGTGAAGGGTGAAGAAAAGAATGCCACTGCCCAATTTTATGCTTAAT
AAGCCGCCAAAATTCAACAGGGTACCGACTTGGCCGAAGTAGCCCCAATATTATGTGCTGGTGTTACTGTATATAAAGCACTAAAAGAGGCAGACTTGAA


TATAボックス5'-TATA[A/T]AA[G/A]-3' を TATA = re.compile('TATA[AT]AA[GA]') としてコンパイルしている。これにより、効率的な処理が実現できる。コンパイルしなくても、re.search('TATA[AT]AA[GA]',　文字列) とすることで、同様の処理が実現できる。　　
　　
次に、pattern.match(文字列) として、正規表現パターンを検索している。この場合、計4種類のTATAボックス配列を検索している。__マッチオブジェクト__ から一致箇所の情報を抽出できる。

__一致箇所を出力する探索__  
TATAボックスを有する配列をreモジュールによる正規表現パターンを用いて探索する。一致箇所を出力する。

In [43]:
TATA = re.compile('TATA[AT]AA[GA]') #パターンをコンパイル AorT  GorA

with open(path) as f:
    for line in f:
        line = line.rstrip()  #改行を削除
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
        s= line.split('\t') #タブで区切る
        sequence = s[9] #10列目を格納

        m = TATA.search(sequence) #pattern を検索

        if m:
            #一致した箇所を出力
            print(m.start(), m.end(), m.string[m.start():m.end()])
        

54 62 TATATAAG
54 62 TATATAAA
45 53 TATAAAAA
69 77 TATATAAA


マッチオブジェクトを用いることで、一致した箇所の情報や一致した文字列の抽出が可能となる。m.start(), m.end()はマッチした部分文字列の先頭と末尾のインデックスを返す。

__相補鎖も含めた探索__  
TATAボックスを有する配列をreモジュールによる正規表現パターンを用いて探索する。一致箇所を出力する。相補鎖についても探索する。

In [44]:
TATA = re.compile('TATA[AT]AA[GA]') #パターンをコンパイル AorT  GorA
TATA_rev = re.compile('[CT]TT[AT]TATA') #相補鎖パターンをコンパイル

with open(path) as f:
    for line in f:
        line = line.rstrip()  #改行を削除
        if line.startswith('@'): #ヘッダー行をスキップ
            continue
        s= line.split('\t') #タブで区切る
        sequence = s[9] #10列目を格納

        m = TATA.search(sequence) #pattern を検索

        if m:
            #一致した箇所を出力
            print(m.start(), m.end(), m.string[m.start():m.end()])

        m = TATA_rev.search(sequence) #相補鎖pattern を検索

        if m:
            #一致した箇所を出力
            print(m.start(), m.end(), m.string[m.start():m.end()])

54 62 TATATAAG
14 22 TTTTTATA
54 62 TATATAAA
89 97 TTTTTATA
62 70 TTTTTATA
9 17 TTTTTATA
45 53 TATAAAAA
69 77 TATATAAA
75 83 TTTTTATA


配列内にパターンが複数含まれる場合には、findall()やfinditer()によって探索が可能となる。searchは、配列内の最初に出現した一致箇所の情報を取得する。

## 5.5 おわりに  
5章では、文字列処理の基本について概説した。GFFやSAMファイルを例にして、ファイルの読み込み/書き込みから情報抽出までを紹介した。これらを駆使すれば、さまざまなファイルから、必要な情報の整理/収集が効率的に実現できる。少しでも研究の幅が広がる一助になれば幸いである。